# pandas中的变形操作

在数据分析中，经常需要对数据进行重塑或转换，这就涉及到了长宽表的变换。在pandas中，我们可以使用各种变形操作来重组数据，使其更适合分析。下面将介绍一些常用的变形操作。

In [18]:
import numpy as np
import pandas as pd

## 长宽表的变形

长表（long format）和宽表（wide format）是数据表格的两种展示形式。长表格式通常具有更多的行，而较少的列；相反，宽表格式具有更多的列，而较少的行。长表格式适合存储重复测量的数据，而宽表格式则便于比较不同变量。

### 长表示例

假设我们有一组学生的考试成绩数据，其中每个学生参加了数学和英语两门课的考试。在长表（Long Format）中，我们将每个学生的每门课成绩作为单独的记录行，这样，每位学生会在表中占据多行，但每行只包含一门课的成绩。这种格式便于表示每个学生的所有成绩记录。

In [19]:
long_df = pd.DataFrame({
    'Student': ['Alice', 'Alice', 'Bob', 'Bob'],
    'Subject': ['Math', 'English', 'Math', 'English'],
    'Score': [85, 92, 78, 80]
})
long_df

,Student,Subject,Score
0,Alice,Math,85
1,Alice,English,92
2,Bob,Math,78
3,Bob,English,80


### 宽表示例

在宽表（Wide Format）中，每位学生的所有成绩都在同一行中，每门课成绩作为单独的列。这种格式使得对每位学生的不同课程成绩进行比较更为直观，但可能使表格的宽度增加，尤其是当涉及多个科目时。

In [20]:
wide_df = pd.DataFrame({
    'Student': ['Alice', 'Bob'],
    'Math Score': [85, 78],
    'English Score': [92, 80]
})
wide_df

,Student,Math Score,English Score
0,Alice,85,92
1,Bob,78,80


长宽表只是展示相同信息的两种不同方式。在实际应用中，长表格式更适合用于长期的数据收集和追踪，例如时间序列数据，而宽表格式在进行特定类型的数据分析，如比较不同科目间的分数时更为方便。

### pivot：长表转宽表

`pivot`函数可以将长表转换为宽表。考虑以下例子，我们有一个包含学生分数的长表格，我们希望将不同的科目分数移至不同的列。

In [21]:
df = pd.DataFrame({
    'Class':[1,1,2,2],
    'Name':['San Zhang','San Zhang','Si Li','Si Li'],
    'Subject':['Chinese','Math','Chinese','Math'],
    'Grade':[80,75,90,85]
})
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Math,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


使用`pivot`函数，我们可以将`Subject`列的不同值转换为不同的列。

In [22]:
wide_df = df.pivot(index='Name', columns='Subject', values='Grade')
wide_df

Subject,Chinese,Math
Name,,
San Zhang,80,75
Si Li,90,85


`pivot`函数需要三个参数：`index`（新表的行索引），`columns`（要转换为列的值），`values`（填充到表中的数据）。在这个例子中，每个学生的名字成为了行索引，科目成为了列，分数则填充到相应的位置。

注意，使用`pivot`时，指定为`index`和`columns`的列组合必须是唯一的，否则pandas不知道如何进行转换，例如：

In [24]:
df.loc[1, 'Subject'] = 'Chinese'
df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,1,San Zhang,Chinese,75
2,2,Si Li,Chinese,90
3,2,Si Li,Math,85


In [25]:
try:
    df.pivot(index='Name', columns='Subject', values='Grade')
except ValueError as e:
    print(e)

Index contains duplicate entries, cannot reshape


### pivot_table：聚合长表转宽表

当长表中存在重复的行列对时，我们需要使用聚合函数来解决冲突，`pivot_table`可以实现这一功能。

In [26]:
df = pd.DataFrame({
    'Name':['San Zhang', 'San Zhang', 'San Zhang', 'San Zhang',
            'Si Li', 'Si Li', 'Si Li', 'Si Li'],
    'Subject':['Chinese', 'Chinese', 'Math', 'Math',
               'Chinese', 'Chinese', 'Math', 'Math'],
    'Grade':[80, 90, 100, 90, 70, 80, 85, 95]
})
df

,Name,Subject,Grade
0,San Zhang,Chinese,80
1,San Zhang,Chinese,90
2,San Zhang,Math,100
3,San Zhang,Math,90
4,Si Li,Chinese,70
5,Si Li,Chinese,80
6,Si Li,Math,85
7,Si Li,Math,95


In [11]:
df.pivot_table(index='Name', columns='Subject', values='Grade', aggfunc='mean')

Subject,Chinese,Math
Name,,
San Zhang,85,95
Si Li,75,90


在这个例子中，我们计算了每个学生在每个科目上的平均分数。`aggfunc`参数指定了聚合函数。

### melt：宽表转长表

`melt`函数可以将宽表转换为长表。考虑以下宽表：

In [27]:
df = pd.DataFrame({
    'Class':[1,2],
    'Name':['San Zhang', 'Si Li'],
    'Chinese':[80, 90],
    'Math':[80, 75]
})
df

,Class,Name,Chinese,Math
0,1,San Zhang,80,80
1,2,Si Li,90,75


我们可以用`melt`把它转换回长表：

In [28]:
long_df = df.melt(id_vars=['Class', 'Name'], value_vars=['Chinese', 'Math'], var_name='Subject', value_name='Grade')
long_df

,Class,Name,Subject,Grade
0,1,San Zhang,Chinese,80
1,2,Si Li,Chinese,90
2,1,San Zhang,Math,80
3,2,Si Li,Math,75


在这里，`id_vars`定义了在转换过程中保持不变的列，`value_vars`定义了要转换的列。`var_name`和`value_name`分别定义了转换后列的名称。